In [ ]:
import os
import numpy as np
from improc.io import parse_collection, DCAccessor
DCAccessor.register()
from holoviews import opts

from inter_view.dashboards import AnnotationDashBoard

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# config

In [ ]:
# set the base directory were training images are located (with raw|annot sbu folders)

basedir = 'annot/path'

data_pattern = '{subdir}/{fname}.{ext}'
index = ['subdir', 'fname']

channel_config = {'raw':{'cmap':'gray', 'intensity_bounds':(200,18500)},
                  'annot':{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'intensity_bounds':(-2**15,2**15-1), 'slider_limits':(-2**15,2**15-1), 'bitdepth':'int16', 'opacity':0.5},}

def set_inactive_tool(plot, element):
    plot.state.toolbar.active_inspect = None

opts.defaults(opts.Image('channel.raw', frame_width=1000),
              opts.Image('channel.annot', frame_width=1000, clipping_colors={'min': (0, 0, 0, 0)}, clim=(0,2**16-1), hooks=[set_inactive_tool], tools=['hover'], show_title=False),
             )

# opts.defaults(opts.Image('channel.segm', clipping_colors={'min': (0, 0, 0, 0)}, clim=(1,256*256-1), tools=['hover']))

# parse files

In [ ]:
df = parse_collection(os.path.join(basedir, data_pattern), index)
df = df.dc[['raw', 'annot']]

df

# interactive dashboard

In [ ]:
from skimage.io import imread

def read_img(path):
    img = imread(path)
    
    # special handling of separator input
    # --> show the mean of 4 channels
    if img.shape[-1] <= 4:
        img = img.astype(np.float32)
        img -= np.quantile(img, 0.001, axis=(0, 1), keepdims=True)
        img /= np.quantile(img, 0.999, axis=(0, 1), keepdims=True)
        img = img.mean(axis=-1)
        img = (img*20000).astype(np.uint16)
    
    return img

db = AnnotationDashBoard(df=df,
                         multi_select_levels=['subdir'],
                         loading_fun=read_img,
                         channel_config=channel_config,
                         composite_channels=['raw'],
                         overlay_channels=['annot'],
                         annot_channel='annot')

db.panel().servable()